In [1]:
!nvidia-smi

Mon Nov 27 12:55:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.1 --progress-bar off
!pip install -qqq transformers==4.35.2 --progress-bar off
!pip install -qqq accelerate==0.24.1 --progress-bar off
!pip install -qqq vllm==0.2.2 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.1.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [3]:
from pathlib import Path

from huggingface_hub import notebook_login
from tqdm import tqdm

notebook_login()

In [1]:
import torch
from transformers import PreTrainedTokenizer, pipeline

text_pipeline = pipeline(
    "text-generation",
    model="meta-llama/Llama-2-7b-chat-hf",
    torch_dtype=torch.float16,
    device_map="auto",
)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## Prompt Preparation

In [2]:
texts = [
    "What are the pros/cons of ChatGPT vs Open Source LLMs?",
    "Write an email to a new client to offer a subscription for a paper supply for 1 year.",
    "I have $10,000 USD for investment. How one should invest it during times of high inflation and high mortgate rates?",
    "Write a function in python that calculates the square of a sum of two numbers.",
]

In [3]:
def create_prompt(text: str, tokenizer: PreTrainedTokenizer) -> str:
    messages = [
        {
            "role": "system",
            "content": "You are a friendly chatbot who always responds as superhuman intelligence AI",
        },
        {"role": "user", "content": text},
    ]
    return tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

In [4]:
prompts = [create_prompt(text, text_pipeline.tokenizer) for text in texts]

In [6]:
for prompt in prompts:
    print(prompt)
    print()
    print()

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI
<</SYS>>

What are the pros/cons of ChatGPT vs Open Source LLMs? [/INST]


<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI
<</SYS>>

Write an email to a new client to offer a subscription for a paper supply for 1 year. [/INST]


<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI
<</SYS>>

I have $10,000 USD for investment. How one should invest it during times of high inflation and high mortgate rates? [/INST]


<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI
<</SYS>>

Write a function in python that calculates the square of a sum of two numbers. [/INST]




In [14]:
folder_path = Path("prompts")
folder_path.mkdir(parents=True, exist_ok=True)

for i, prompt in enumerate(prompts, start=1):
    file_path = folder_path / f"prompt_{i}.txt"
    file_path.write_text(prompt)

## HF Inference

In [5]:
%%time
outputs = text_pipeline(prompts[0], max_new_tokens=256, do_sample=True, temperature=0.1)

CPU times: user 16.9 s, sys: 416 ms, total: 17.3 s
Wall time: 20.6 s


In [6]:
print(outputs[0]["generated_text"])

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI
<</SYS>>

What are the pros/cons of ChatGPT vs Open Source LLMs? [/INST]  Greetings! I'm here to help you compare ChatGPT and Open Source LLMs. Both are powerful language models, but they have some differences in their architecture, training data, and usage. Here are some pros and cons of each:

ChatGPT:

Pros:

1. High-quality responses: ChatGPT is trained on a massive dataset of text from the internet, which allows it to generate high-quality responses to a wide range of prompts.
2. Customizable: ChatGPT can be fine-tuned on specific domains or tasks, making it highly customizable for various applications.
3. Easy integration: ChatGPT can be easily integrated into various platforms and applications, such as messaging apps, chatbots, and virtual assistants.
4. Fast response times: ChatGPT can respond quickly to user input, making it suitable for real-time applications.

Cons:

1. Limited dom

## vLLM Inference

In [4]:
from pathlib import Path

import torch
from vllm import LLM, SamplingParams

In [6]:
folder_path = Path("prompts")
prompts = []

for file_path in sorted(list(folder_path.glob("*.txt"))):
    content = file_path.read_text()
    prompts.append(content)

In [7]:
print(prompts[0])

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI
<</SYS>>

What are the pros/cons of ChatGPT vs Open Source LLMs? [/INST]


In [8]:
llm = LLM(model="meta-llama/Llama-2-7b-chat-hf", dtype=torch.float16)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

INFO 11-27 13:00:17 llm_engine.py:72] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)
INFO 11-27 13:00:17 tokenizer.py:31] For some LLaMA V1 models, initializing the fast tokenizer may take a long time. To reduce the initialization time, consider using 'hf-internal-testing/llama-tokenizer' instead of the original tokenizer.


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

INFO 11-27 13:03:43 llm_engine.py:207] # GPU blocks: 26, # CPU blocks: 512


In [9]:
sampling_params = SamplingParams(temperature=0.1, max_tokens=256)

In [10]:
%%time
outputs = llm.generate(prompts[0], sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:14<00:00, 14.10s/it]

CPU times: user 13.8 s, sys: 5.28 ms, total: 13.9 s
Wall time: 14.1 s


In [11]:
for output in outputs:
    generated_text = output.outputs[0].text
    print(generated_text)

  Greetings! I'm here to help you compare the pros and cons of ChatGPT and Open Source LLMs. Both are powerful language models, but they have different strengths and weaknesses. Here's a comprehensive comparison:

ChatGPT:
Pros:

1. High-quality responses: ChatGPT is trained on a massive dataset and has been fine-tuned for a wide range of tasks, resulting in high-quality responses that are often indistinguishable from human language.
2. Easy to use: ChatGPT has a simple and intuitive API that makes it easy to integrate into various applications, such as chatbots, voice assistants, and language translation tools.
3. Fast response times: ChatGPT can generate responses quickly, even for long conversations, thanks to its parallel processing capabilities.
4. Customizable: ChatGPT can be fine-tuned for specific tasks and domains, allowing developers to tailor the model to their needs.

Cons:

1. Limited domain knowledge: While ChatGPT has been trained on a vast dataset, its knowledge is limi

## Batch Inference

In [7]:
%%time
hf_batch_outputs = text_pipeline(
    prompts, max_new_tokens=256, do_sample=True, temperature=0.1
)

CPU times: user 1min 2s, sys: 115 ms, total: 1min 2s
Wall time: 1min 14s


In [8]:
for output in hf_batch_outputs:
    print(output[0]["generated_text"])
    print()

<s>[INST] <<SYS>>
You are a friendly chatbot who always responds as superhuman intelligence AI
<</SYS>>

What are the pros/cons of ChatGPT vs Open Source LLMs? [/INST]  Greetings! I'm here to help you compare ChatGPT and Open Source LLMs. Both are powerful language models, but they have different strengths and weaknesses. Here are some pros and cons of each:

ChatGPT:
Pros:

1. High-quality responses: ChatGPT is trained on a massive dataset and has been fine-tuned for a wide range of tasks, resulting in high-quality responses that are often indistinguishable from those generated by humans.
2. Easy to use: ChatGPT has a user-friendly interface that makes it easy to integrate into various applications, such as chatbots, virtual assistants, and language translation tools.
3. Scalability: ChatGPT can handle a large volume of requests simultaneously, making it a great choice for applications that require fast and efficient responses.
4. Customization: ChatGPT can be customized to fit the sp

In [12]:
%%time
vllm_batch_outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 4/4 [00:42<00:00, 10.73s/it]

CPU times: user 41.2 s, sys: 0 ns, total: 41.2 s
Wall time: 42.9 s


In [13]:
for output in vllm_batch_outputs:
    generated_text = output.outputs[0].text
    print(generated_text)
    print()

  Greetings! I'm here to help you compare ChatGPT and Open Source LLMs. Both are powerful language models, but they have some differences that may make one more suitable for your needs than the other. Here are some pros and cons of each:

ChatGPT:

Pros:

1. High-quality responses: ChatGPT is trained on a massive dataset and has been fine-tuned for a wide range of tasks, including conversational dialogue. This means it can generate high-quality responses that are contextually appropriate and coherent.
2. Easy to use: ChatGPT has a user-friendly interface that makes it simple to integrate into your application or platform. You can easily interact with it using natural language inputs, and it can respond in a conversational tone.
3. Customizable: ChatGPT can be customized to fit your specific needs by fine-tuning the model on your own dataset or using specific prompts and responses.
4. Fast response times: ChatGPT can respond quickly to user inputs, making it ideal for real-time applicat

## References

- [vLLM Docs](https://docs.vllm.ai/en/latest/)